<p>
    <a href="https://colab.research.google.com/github/explodinggradients/ragas/blob/main/docs/quickstart.ipynb">
        <img alt="Open In Colab" 
             align="left"
             src="https://colab.research.google.com/assets/colab-badge.svg">
    </a>
    <br>
    <h1> Quickstart </h1>
</p>

welcome to the ragas quickstart. We're going to get you up and running with ragas as qickly as you can so that you can go back to improving your Retrieval Augmented Generation pipelines while this library makes sure your changes are improving your entire pipeline.

to kick things of lets start with the data

In [1]:
# if using colab uncomment this
#!pip install ragas

Ragas also uses OpenAI for running some metrics so make sure you have your openai key ready and available in your environment

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-key"

## The Data

Ragas performs a `ground_truth` free evaluation of your RAG pipelines. This is because for most people building a gold labeled dataset which represents in the distribution they get in production is a very expensive process.

Hence to work with ragas all you need are the following data
- question: `list[str]` - These are the questions you RAG pipeline will be evaluated on. 
- answer: `list[str]` - The answer generated from the RAG pipeline and give to the user.
- contexts: `list[list[str]]` - The contexts which where passed into the LLM to answer the question.

Ideally your list of questions should reflect the questions your users give, including those that you have been problamatic in the past.

Here we're using an example dataset from on of the baselines we created for the [Financial Opinion Mining and Question Answering (fiqa) Dataset](https://sites.google.com/view/fiqa/) we created. If you want to want to know more about the baseline, feel free to check the `experiements/baseline` section

In [3]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

Found cached dataset fiqa (/home/jjmachan/.cache/huggingface/datasets/explodinggradients___fiqa/ragas_eval/1.0.0/3dc7b639f5b4b16509a3299a2ceb78bf5fe98ee6b5fee25e7d5e4d290c88efb8)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

## Metrics

Ragas measures your pipeline's performance against two dimensions

1. Faithfulness: measures the factual consistency of the generated answer against the given context.
2. Relevancy: measures how relevant retrieved contexts and the generated answer are to the question.

Through repeated experiments, we have found that the quality of a RAG pipeline is highly dependent on these two dimensions. The final `ragas_score` is the harmonic mean of these two factors.

now lets import these metrics and understand more about what they denote

In [8]:
from ragas.metrics import context_relevancy, answer_relevancy, faithfulness
from ragas.metrics.critique import harmfulness

here you can see that we are using 4 metrics, but what do the represent?

1. context_relevancy - a measure of how relevent the retrieved context is to the question. Conveys quality of the retrieval pipeline.
2. answer_relevancy - a measure of how relevent the answer is to the question
3. faithfulness - the factual consistancy of the answer to the context base on the question.
4. harmfulness (AspectCritique) - in general, `AspectCritique` is a metric that can be used to quantify various aspects of the answer. Aspects like harmfulness, maliciousness, coherence, correctness, concisenes are available by default but you can easily define your own. Check the [docs](./metrics.md) for more info.

**Note:** *by default these metrics are using OpenAI's API to compute the score. If you using this metric make sure you set the environment key `OPENAI_API_KEY` with your API key. You can also try other LLMs for evaluation, check the [llm guide](./guides/llms.ipynb) to learn more*

If you're interested in learning more, feel free to check the [docs](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

## Evaluation

Running the evalutation is as simple as calling evaluate on the `Dataset` with the metrics of your choice.

In [9]:
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"],
    metrics=[context_relevancy, faithfulness, answer_relevancy, harmfulness],
)

result

evaluating with [context_ relevancy]


100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.05s/it]


evaluating with [faithfulness]


100%|████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.11s/it]


evaluating with [answer_relevancy]


100%|████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.20s/it]


evaluating with [harmfulness]


100%|████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.75s/it]


{'ragas_score': 0.1787, 'context_ relevancy': 0.0689, 'faithfulness': 0.8333, 'answer_relevancy': 0.9347, 'harmfulness': 0.0000}

and there you have the it, all the scores you need. `ragas_score` gives you a single metric that you can use while the other onces measure the different parts of your pipeline.

now if we want to dig into the results and figure out examples where your pipeline performed worse or really good you can easily convert it into a pandas array and use your standard analytics tools too!

In [10]:
df = result.to_pandas()
df.head()

,question,ground_truths,answer,contexts,context_ relevancy,faithfulness,answer_relevancy,harmfulness
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,0.132468,1.0,0.978180,0
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,0.074175,1.0,0.909481,0
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.000000,0.5,0.916480,0


And thats it!

You can check out the [ragas in action] notebook to get a feel of what is like to use it while trying to improve your pipelines.

if you have any suggestion/feedbacks/things your not happy about, please do share it in the [issue section](https://github.com/explodinggradients/ragas/issues). We love hearing from you 😁